# Scraping News Articles

In [1]:
# Library
import pandas as pd #df
import matplotlib.font_manager as fm # font
import matplotlib.pyplot as plt # font
import requests # for scraping
from bs4 import BeautifulSoup # for scraping
from newspaper import Article
from konlpy.tag import Mecab, Hannanum, Kkma, Komoran, Okt # 형태소
# machine learning
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.feature_extraction.text import HashingVectorizer

## 데이터 import

### naver news article 직접 수집

#### URL 수집하는 함수
- 원하는 페이지수(page_num), 카테고리 번호(code), 날짜(date) 입력   
-예를 들어 2020년 8월 3일 경제기사(코드 101)를 2페이지까지 받고 싶으면, make_urllist(2,101,20200803)

In [ ]:
def make_urllist(page_num,code,date):
    urllist=[]
    for i in range(1,page_num+1):
        url='https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1='+str(code)+'&date='+str(date)+'&page='+str(i)   
        news = requests.get(url)
        
        # BeautifulSoup의 인스턴스 생성. 
        soup = BeautifulSoup(news.content,'html.parser')
        
        # CASE 1
        news_list = soup.select('.newsflash_body .type06_headline li dl')
        # CASE 2
        news_list.extend(soup.select('.newsflash_body .type06 li dl'))
        
    # 각 뉴스로부터 a 태그인 <a href ='주소'> 에서 '주소'만을 가져옵니다.
        for line in news_list:
            urllist.append(line.a.get('href'))
    return urllist       

#### 받아온 url이용하여 기사 저장
- url list와 분류를 이용하여 기사 내용 저장


In [ ]:
# 뉴스기사 분류 코드
idx2word = {'101' : '경제', '102' : '사회', '103' : '생활/문화', '105' : 'IT/과학'}

#- 데이터프레임을 생성하는 함수입니다.
def make_data(urllist, code):
    text_list = []
    for url in urllist:
        article = Article(url, language='ko')
        article.download()
        article.parse()
        text_list.append(article.text)
    
    #- 데이터프레임의 'news' 키 아래 파싱한 텍스트를 밸류로 붙여줍니다.
    df = pd.DataFrame({'news': text_list})

    #- 데이터프레임의 'code' 키 아래 한글 카테고리명을 붙여줍니다.
    df['code'] = idx2word[str(code)]
    return df

#### 여러 카테고리의 기사 가져오기

In [ ]:
# 페이지 수, code_list, 날짜 기준으로 수집하는 함수
def make_total_data(page_num, code_list, date):
    df = None
    
    for code in code_list:
        url_list = make_urllist(page_num, code, date)
        df_temp = make_data(url_list, code)
        print(str(code)+'번 코드에 대한 데이터를 만들었습니다.')
        
    if df is not None:
        df = pd.concat([df, df_temp])
    else:
        df = df_temp
    
    return df

In [ ]:
# df = make_total_data(100, code_list, 20200803)

### 미리 수집된 뉴스 기사 데이터 사용

In [2]:
df =pd.read_csv('/home/aiffel0036/aiffel/data_represent/data/news_data.csv')
df2 =pd.read_csv('/home/aiffel0036/aiffel/data_represent/data/news_data2.csv')

df = pd.concat([df2,df], axis=0)

## 데이터 탐색 및 정제

In [3]:
# head, null, length 확인하는 function
def data_exp(df):
    print("head출력: \n", df.head())
    print("null값 확인: \n", df.isnull().sum())
    print("length: \n", len(df))

In [4]:
data_exp(df)

head출력: 
                                                 news code
0  기사 섹션 분류 안내\n\n기사의 섹션 정보는 해당 언론사의 분류를 따르고 있습니다...   경제
1  ▶제21대 총선 실시간 개표 현황 및 결과 보기\n\n총선에서 여당이 다시 한 번 ...   경제
2  [뉴욕=AP/뉴시스]지난 10일 뉴욕 증권거래소 건물에 미국 국기가 게양되어 있다....   경제
3  부산지역 주유소에서 판매하는 기름값이 휘발유는 평균 1200원대, 경유는 1000원...   경제
4  담배업계가 소비자의 취향을 저격한 다양한 담배 신제품들을 잇달아 선보이고 있다.이전...   경제
null값 확인: 
 news    0
code    0
dtype: int64
length: 
 8827


### 중복값 확인 및 제거

In [8]:
# 중복 값 확인 => 중복으로 수집되고 있기 때문
df[df.duplicated(subset=['news'], keep = 'first')].sort_values(by=['news'])
#df2[df2.duplicated(subset=['news'], keep = 'first')].sort_values(by=['news'])
# 중복된 샘플들을 제거
df.drop_duplicates(subset=['news'], inplace = True)
#df2.drop_duplicates(subset=['news'], inplace = True)
print("length: \n", len(df))
df.head()
# 확인해보니, 0번째는 기사가 아님 -> 삭제
df = df[1:]

length: 
 6127


### 한글 외의 문자는 제거

In [9]:
# 정규 표현식을 이용해서 한글 외의 문자는 전부 제거합니다.
df['news'] = df['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
#df2['news'] = df2['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df['news']

1       제대 총선 실시간 개표 현황 및 결과 보기총선에서 여당이 다시 한 번 승리를 거두면...
2       뉴욕뉴시스지난 일 뉴욕 증권거래소 건물에 미국 국기가 게양되어 있다 신종 코로나바이...
3       부산지역 주유소에서 판매하는 기름값이 휘발유는 평균 원대 경유는 원대에 돌입했다일 ...
4       담배업계가 소비자의 취향을 저격한 다양한 담배 신제품들을 잇달아 선보이고 있다이전까...
6       이데일리 박태진 기자 다음은 일자 이데일리신문 주요 기사다면민주 연승 자만 말고 협...
                              ...                        
4825    신종 코로나바이러스 감염증코로나 사태 이후 가정의 달 월에도 언택트비대면 신풍속도가...
4826    는 소비자로부터 월 이용료 만만원을 받고 초고속 인터넷을 제공한다 그런 브로드밴드가...
4827    머리를 긁고 있는 오랑우탄 몸을 긁는 행동을 따라 하는 것은 부정적 감정과 관련이 ...
4828    가 오는 일 정식 출시하는 스마트폰 벨벳이 사실상 공짜폰이 될 전망이다 단말기 가격...
4829    이미지제공게티이미지뱅크 이미지제공게티이미지뱅크  전자신문  전자신문인터넷 무단전재 ...
Name: news, Length: 6126, dtype: object

### 카테고리별 분포 확인

In [10]:
print(df['code'].value_counts())
#print(df2['code'].value_counts())
# 경제 > 사회 > 생활/문화 > IT/과학

사회       2221
생활/문화    1866
IT/과학    1138
경제        901
Name: code, dtype: int64


### Tokenization
- 문자열을 특정 단위(token)으로 나누어 분석을 하게 됨.
- 한국어의 경우 형태소 분석기를 이용하여 토큰화를 함.

#### Eliminating stopwards
- 자연어 처리에 불필요한 단어들. 한국어의 경우 조사, 접사가 이에 해당.
- 미리 한번에 정의하기보다는 tokenization을 통해 여러 차례 확인하며 제거

In [52]:
# 조사 어미 파일 import
josa = pd.read_csv('/home/aiffel0036/aiffel/data_represent/data/JOSA2.TXT', sep=" ", header=None)
eomi = pd.read_csv('/home/aiffel0036/aiffel/data_represent/data/EOMI2.TXT', sep=" ", header=None)
# to list
josa = josa[0].values.tolist()
eomi = eomi[0].values.tolist()
# merge
stopwords = josa + eomi + ['에','는','은','을','했','에게','있','이','의','하','한','다','과','때문','할','수','무단','따른','및','금지','전재','경향신문','기자','는데','가','등','들','파이낸셜','저작','등','뉴스','동영상']
len(stopwords)
#stopwords = ['에','는','은','을','했','에게','있','이','의','하','한','다','과','때문','할','수','무단','따른','및','금지','전재','경향신문','기자','는데','가','등','들','파이낸셜','저작','등','뉴스']

1203

In [12]:
# 토큰화 및 토큰화 과정에서 불용어를 제거하는 함수입니다.
def preprocessing(data,tokenizer):
    text_data = []
    
    for sentence in data:
        temp_data = []
        #- 토큰화
        temp_data = tokenizer.morphs(sentence) 
        #- 불용어 제거
        temp_data = [word for word in temp_data if not word in stopwords] 
        text_data.append(temp_data)
    
    text_data = list(map(' '.join, text_data))

    return text_data

In [13]:
token_Mecab = Mecab()
token_Hannanum = Hannanum()
token_Kkma = Kkma()
token_Komoran = Komoran()
token_Okt = Okt()

In [47]:
%time text_Mecab = preprocessing(df['news'], token_Mecab)
#text_Mecab2 = preprocessing(df2['news'], token_Mecab)

CPU times: user 5.64 s, sys: 40.1 ms, total: 5.68 s
Wall time: 5.68 s


In [22]:
%time text_Hannanum = preprocessing(df['news'], token_Hannanum)

In [23]:
# 다시 실행시키기 무서움 => 오래걸림
text_Kkma = preprocessing(df['news'], token_Kkma)

In [24]:
%time text_Komoran = preprocessing(df['news'], token_Komoran)

CPU times: user 1min 43s, sys: 222 ms, total: 1min 43s
Wall time: 1min 30s


In [25]:
%time text_Okt = preprocessing(df['news'], token_Okt)

CPU times: user 40min 29s, sys: 986 ms, total: 40min 30s
Wall time: 5min 57s


## Model 

### Splitting data into train and test set

In [48]:
X_train, X_test, y_train, y_test = train_test_split(text_Mecab, df['code'],test_size=0.2, shuffle=True, random_state = 34)
print('훈련용 뉴스 기사의 개수 :', len(X_train))
print('테스트용 뉴스 기사의 개수 : ', len(X_test))
print('훈련용 레이블의 개수 : ', len(y_train))
print('테스트용 레이블의 개수 : ', len(y_test))

훈련용 뉴스 기사의 개수 : 4900
테스트용 뉴스 기사의 개수 :  1226
훈련용 레이블의 개수 :  4900
테스트용 레이블의 개수 :  1226


In [32]:
X_train, X_test, y_train, y_test = train_test_split(text_Hannanum, df['code'],test_size=0.2, shuffle=True, random_state = 34)
print('훈련용 뉴스 기사의 개수 :', len(X_train))
print('테스트용 뉴스 기사의 개수 : ', len(X_test))
print('훈련용 레이블의 개수 : ', len(y_train))
print('테스트용 레이블의 개수 : ', len(y_test))

훈련용 뉴스 기사의 개수 : 4900
테스트용 뉴스 기사의 개수 :  1226
훈련용 레이블의 개수 :  4900
테스트용 레이블의 개수 :  1226


In [36]:
X_train, X_test, y_train, y_test = train_test_split(text_Kkma, df['code'],test_size=0.2, shuffle=True, random_state = 34)
print('훈련용 뉴스 기사의 개수 :', len(X_train))
print('테스트용 뉴스 기사의 개수 : ', len(X_test))
print('훈련용 레이블의 개수 : ', len(y_train))
print('테스트용 레이블의 개수 : ', len(y_test))

훈련용 뉴스 기사의 개수 : 4900
테스트용 뉴스 기사의 개수 :  1226
훈련용 레이블의 개수 :  4900
테스트용 레이블의 개수 :  1226


In [40]:
X_train, X_test, y_train, y_test = train_test_split(text_Komoran, df['code'],test_size=0.2, shuffle=True, random_state = 34)
print('훈련용 뉴스 기사의 개수 :', len(X_train))
print('테스트용 뉴스 기사의 개수 : ', len(X_test))
print('훈련용 레이블의 개수 : ', len(y_train))
print('테스트용 레이블의 개수 : ', len(y_test))

훈련용 뉴스 기사의 개수 : 4900
테스트용 뉴스 기사의 개수 :  1226
훈련용 레이블의 개수 :  4900
테스트용 레이블의 개수 :  1226


In [44]:
X_train, X_test, y_train, y_test = train_test_split(text_Okt, df['code'],test_size=0.2, shuffle=True, random_state = 34)
print('훈련용 뉴스 기사의 개수 :', len(X_train))
print('테스트용 뉴스 기사의 개수 : ', len(X_test))
print('훈련용 레이블의 개수 : ', len(y_train))
print('테스트용 레이블의 개수 : ', len(y_test))

훈련용 뉴스 기사의 개수 : 4900
테스트용 뉴스 기사의 개수 :  1226
훈련용 레이블의 개수 :  4900
테스트용 레이블의 개수 :  1226


### TF-IDF vector로 변환
- Term Frequency * Inverse Document Frequency의 약자
- 단어가 많이 등장할 수록 관련이 많을 것이다라고 생각.
- 하지만, 연관성 없이 여러번 나타날 때 문제가 됨.
- 따라서 패널티 주기위해 IDF 사용. Log (Total number of Docs / number of Docs)

In [20]:
# 텍스트를 입력하면 자동으로 TF-IDF vector로 바꾸어주는 전처리 함수
def tfidf_vectorizer(data):
    data_counts = count_vect.transform(data)
    data_tfidf = tfidf_transformer.transform(data_counts)
    return data_tfidf

In [49]:
# 텍스트 데이터 벡터로 변환해야 함.
# 단어의 수를 카운트하는 사이킷런의 카운트벡터라이저.
count_vect = CountVectorizer()
#print(count_vect.vocabulary_)
# document-form matrix로 변환
X_train_counts = count_vect.fit_transform(X_train)

#- 카운트벡터라이저의 결과로부터 TF-IDF 결과를 얻습니다.
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

### 나이브 베이즈 분류기

In [50]:
#- 나이브 베이즈 분류기를 수행합니다.
#- X_train은 TF-IDF 벡터, y_train은 레이블입니다.
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [51]:
# mecab
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       IT/과학       0.84      0.73      0.78       224
          경제       0.91      0.47      0.62       171
          사회       0.73      0.94      0.82       446
       생활/문화       0.79      0.76      0.77       385

    accuracy                           0.78      1226
   macro avg       0.82      0.72      0.75      1226
weighted avg       0.79      0.78      0.77      1226



In [35]:
# hannanum
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       IT/과학       0.87      0.68      0.76       224
          경제       0.89      0.47      0.62       171
          사회       0.73      0.93      0.82       446
       생활/문화       0.76      0.77      0.77       385

    accuracy                           0.77      1226
   macro avg       0.81      0.71      0.74      1226
weighted avg       0.79      0.77      0.76      1226



In [39]:
# Kkma
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       IT/과학       0.85      0.72      0.78       224
          경제       0.87      0.55      0.67       171
          사회       0.74      0.94      0.82       446
       생활/문화       0.82      0.76      0.79       385

    accuracy                           0.79      1226
   macro avg       0.82      0.74      0.77      1226
weighted avg       0.80      0.79      0.78      1226



In [43]:
# Komoran
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       IT/과학       0.87      0.73      0.79       224
          경제       0.86      0.54      0.66       171
          사회       0.75      0.94      0.83       446
       생활/문화       0.81      0.78      0.80       385

    accuracy                           0.80      1226
   macro avg       0.82      0.75      0.77      1226
weighted avg       0.81      0.80      0.79      1226



In [45]:
# Okt
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       IT/과학       0.87      0.75      0.81       224
          경제       0.87      0.48      0.62       171
          사회       0.73      0.93      0.82       446
       생활/문화       0.82      0.78      0.80       385

    accuracy                           0.79      1226
   macro avg       0.82      0.74      0.76      1226
weighted avg       0.80      0.79      0.78      1226



## Conclusion
1. Kkma는 너무 오래걸려서 효율성이 떨어지기 때문에 사용하지 않는 것이 좋을 것 같다.
2. f1-score는 Komoran이 가장 높았다.
3. 데이터 전처리에 더 공을 들여야 할 것 같다 -> 아쉽게도 시간이 부족.   
특히, 단어 frequency나 불용어를 더 자세하게 보아야 할 것 같다. 단어들을 하나씩 보면 이상하게 끊긴 것들이 은근 많음.   

- 개선책: f1-score는 다들 비슷하므로 가장 속도가 빠른 Mecab을 이용하고, 데이터 전처리 단계에 더 공을 들일 것